In [1]:
#!git clone https://github.com/milesial/Pytorch-UNet.git
# Fix for cuda init error
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import argparse
import logging
import sys
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

from utils.clothing_dataset import BasicDataset, ClothingDataset
from utils.dice_score import dice_loss
from utils.evaluate import evaluate
from unet import UNet
import gc
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
import torch.cuda
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

True


device(type='cuda')

# Setup wandb

In [4]:
wandb.init(project='unet', entity='endev')

wandb: Currently logged in as: endev (use `wandb login --relogin` to force relogin)


# Dataset Path Variables

In [5]:
root = "./data/"
dir_checkpoint = Path('./training/')

In [6]:
train_dir_img = Path('/data/datasets/clothing-size/train/imgs/')
train_dir_mask = Path('/data/datasets/clothing-size/train/mask/')

In [7]:
val_dir_img = Path('/data/datasets/clothing-size/val/imgs/')
val_dir_mask = Path('/data/datasets/clothing-size/val/mask/')

# Check data

In [8]:
ls /data/datasets/clothing-size/val/imgs/

DSC_2702.jpg*  DSC_2716.jpg*  IMG_9283.jpg*  IMG_9386.jpg*  IMG_9427.jpg*
DSC_2704.jpg*  DSC_2721.jpg*  IMG_9315.jpg*  IMG_9388.jpg*  IMG_9429.jpg*
DSC_2707.jpg*  DSC_2722.jpg*  IMG_9326.jpg*  IMG_9389.jpg*  IMG_9430.jpg*
DSC_2708.jpg*  IMG_9263.jpg*  IMG_9329.jpg*  IMG_9398.jpg*
DSC_2714.jpg*  IMG_9279.jpg*  IMG_9330.jpg*  IMG_9422.jpg*


In [9]:
ls /data/datasets/clothing-size/val/mask/

DSC_2702_mask.gif*  DSC_2721_mask.gif*  IMG_9326_mask.gif*  IMG_9398_mask.gif*
DSC_2704_mask.gif*  DSC_2722_mask.gif*  IMG_9329_mask.gif*  IMG_9422_mask.gif*
DSC_2707_mask.gif*  IMG_9263_mask.gif*  IMG_9330_mask.gif*  IMG_9427_mask.gif*
DSC_2708_mask.gif*  IMG_9279_mask.gif*  IMG_9386_mask.gif*  IMG_9429_mask.gif*
DSC_2714_mask.gif*  IMG_9283_mask.gif*  IMG_9388_mask.gif*  IMG_9430_mask.gif*
DSC_2716_mask.gif*  IMG_9315_mask.gif*  IMG_9389_mask.gif*


In [10]:
def train_net(net,
              device,
              epochs: int = 5,
              batch_size: int = 1,
              learning_rate: float = 0.001,
              save_checkpoint: bool = True,
              img_scale: float = 0.5,
              amp: bool = False):
    # 1. Create dataset
    train_dataset = ClothingDataset(train_dir_img, train_dir_mask, img_scale)
    val_dataset = ClothingDataset(val_dir_img, val_dir_mask, img_scale)

    # 2. Totals
    n_train = len(train_dataset)
    n_val = len(val_dataset)

    # 3. Create data loaders
    loader_args = dict(batch_size=batch_size, num_workers=1, pin_memory=True)
    train_loader = DataLoader(train_dataset, shuffle=True, **loader_args)
    val_loader = DataLoader(val_dataset, shuffle=False, drop_last=True, **loader_args)

    # (Initialize logging)
    experiment = wandb.init(project='U-Net-mask', resume='allow', anonymous='must')
    experiment.config.update(dict(epochs=epochs, 
                                  batch_size=batch_size, 
                                  learning_rate=learning_rate,
                                  save_checkpoint=save_checkpoint, 
                                  img_scale=img_scale,
                                  amp=amp))
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {learning_rate}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_checkpoint}
        Device:          {device.type}
        Images scaling:  {img_scale}
        Mixed Precision: {amp}
    ''')

    # 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    criterion = nn.CrossEntropyLoss()
    global_step = 0

    # 5. Begin training
    for epoch in range(epochs):
        net.train()
        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                images = batch['image']
                true_masks = batch['mask']

                assert images.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {images.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                images = images.to(device=device, dtype=torch.float32)
                true_masks = true_masks.to(device=device, dtype=torch.long)

                with torch.cuda.amp.autocast(enabled=amp):
                    masks_pred = net(images)
                    loss = criterion(masks_pred, true_masks) \
                           + dice_loss(F.softmax(masks_pred, dim=1).float(),
                                       F.one_hot(true_masks, net.n_classes).permute(0, 3, 1, 2).float(),
                                       multiclass=True)

                optimizer.zero_grad(set_to_none=True)
                grad_scaler.scale(loss).backward()
                grad_scaler.step(optimizer)
                grad_scaler.update()

                pbar.update(images.shape[0])
                global_step += 1
                epoch_loss += loss.item()
                experiment.log({
                    'train loss': loss.item(),
                    'step': global_step,
                    'epoch': epoch
                })
                
                pbar.set_postfix(**{'loss (batch)': loss.item()})

                # Evaluation round
                if global_step % (n_train // (2 * batch_size)) == 0:
                    histograms = {}
                    for tag, value in net.named_parameters():
                        tag = tag.replace('/', '.')
                        histograms['Weights/' + tag] = wandb.Histogram(value.data.cpu())
                        histograms['Gradients/' + tag] = wandb.Histogram(value.grad.data.cpu())

                    val_score = evaluate(net, val_loader, device)
                    scheduler.step(val_score)

                    logging.info('Validation Dice score: {}'.format(val_score))
                    experiment.log({
                        'learning rate': optimizer.param_groups[0]['lr'],
                        'validation Dice': val_score,
                        'images': wandb.Image(images[0].cpu()),
                        'masks': {
                            'true': wandb.Image(true_masks[0].float().cpu()),
                            'pred': wandb.Image(torch.softmax(masks_pred, dim=1)[0].float().cpu()),
                        },
                        'step': global_step,
                        'epoch': epoch,
                        **histograms
                    })

        if save_checkpoint:
            Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
            torch.save(net.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}_{}.pth'.format(epoch, datetime.now().strftime("%m_%d_%Y_%H_%M_%S"))))
            logging.info(f'Checkpoint {epoch + 1} saved at {datetime.now().strftime("%m_%d_%Y_%H_%M_%S")}!')

In [11]:
# Number of epochs
epochs = 50

# Batch size
batch_size=1

# Learning rate
lr = 0.0001

# Load model from a .pth file
load = False

# Downscaling factor of the image
scale = 1

# Use mixed precision
amp = False

weight_decay=1e-8

momentum=0.9

config = wandb.config
config.epochs = epochs
config.batch_size = batch_size
config.lr = lr
config.load = load
config.scale = scale
config.amp = amp
config.weight_decay = weight_decay
config.momentum = momentum

config.model = "Unet-mask"

In [12]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [13]:
net = UNet(n_channels=3, n_classes=2, bilinear=True)
net.to(device="cuda")

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [14]:
logging.info(f'Network:\n'
                 f'\t{net.n_channels} input channels\n'
                 f'\t{net.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')

INFO: Network:
	3 input channels
	2 output channels (classes)
	Bilinear upscaling


In [15]:
gc.collect()
torch.cuda.empty_cache()

In [16]:
try:
    train_net(net=net,
              epochs=epochs,
              batch_size=batch_size,
              learning_rate=lr,
              device=device,
              img_scale=scale,
              amp=amp)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    sys.exit(0)

INFO: Creating dataset with 97 examples
INFO: Creating dataset with 23 examples


INFO: Starting training:
        Epochs:          50
        Batch size:      1
        Learning rate:   0.0001
        Training size:   97
        Validation size: 23
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
        Mixed Precision: False
    
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.48batch/s]
                                                                    INFO: Validation Dice score: 0.6776722073554993
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.45batch/s]
                                                                    INFO: Validation Dice score: 0.6884751915931702
Epoch 1/50: 100%|██████████| 97/97 [00:22<00:00,  4.35img/s, loss (batch)=0.14] 
INFO: Checkpoint 1 saved at 10_16_2021_08_49_20!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.52batch/s]
                                                                    INFO: Validation Dice score: 0.6728065013885498
Validation round:  96%|█

Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.32batch/s]
                                                                    INFO: Validation Dice score: 0.6257582306861877
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.31batch/s]
                                                                    INFO: Validation Dice score: 0.6930906176567078
Epoch 8/50: 100%|██████████| 97/97 [00:22<00:00,  4.34img/s, loss (batch)=0.262] 
INFO: Checkpoint 8 saved at 10_16_2021_08_52_00!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.28batch/s]
                                                                    INFO: Validation Dice score: 0.681964635848999
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.22batch/s]
                                                                    INFO: Validation Dice score: 0.5492073893547058
Epoch 9/50: 100%|██████████| 97/97 [00:22<00:00,  4.33img/s, loss (batch)=0.0913]
INFO: Checkpoint 9 saved at 10_16_2021_08_52_23!

Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.10batch/s]
                                                                    INFO: Validation Dice score: 0.5802702307701111
Epoch 15/50: 100%|██████████| 97/97 [00:23<00:00,  4.13img/s, loss (batch)=0.0834]
INFO: Checkpoint 15 saved at 10_16_2021_08_54_45!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.15batch/s]
                                                                    INFO: Validation Dice score: 0.6658838391304016
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 17.08batch/s]
                                                                    INFO: Validation Dice score: 0.6564846038818359
Epoch 16/50: 100%|██████████| 97/97 [00:22<00:00,  4.22img/s, loss (batch)=0.114] 
INFO: Checkpoint 16 saved at 10_16_2021_08_55_08!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.23batch/s]
                                                                    INFO: Validation Dice score: 0.69210433959

Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.26batch/s]
                                                                    INFO: Validation Dice score: 0.6205825209617615
Epoch 22/50: 100%|██████████| 97/97 [00:22<00:00,  4.26img/s, loss (batch)=0.0951]
INFO: Checkpoint 22 saved at 10_16_2021_08_57_34!
Validation round: 100%|██████████| 23/23 [00:01<00:00, 18.17batch/s]
                                                                    INFO: Validation Dice score: 0.5455369353294373
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.24batch/s]
                                                                    INFO: Validation Dice score: 0.6340135931968689
Epoch 23/50: 100%|██████████| 97/97 [00:24<00:00,  3.89img/s, loss (batch)=0.0669]
INFO: Checkpoint 23 saved at 10_16_2021_08_57_59!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.20batch/s]
                                                                    INFO: Validation Dice score: 0.62906032800

Epoch 29/50: 100%|██████████| 97/97 [00:23<00:00,  4.06img/s, loss (batch)=0.125] 
INFO: Checkpoint 29 saved at 10_16_2021_09_00_27!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.12batch/s]
                                                                    INFO: Validation Dice score: 0.5675231218338013
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.24batch/s]
                                                                    INFO: Validation Dice score: 0.6694503426551819
Epoch 30/50: 100%|██████████| 97/97 [00:24<00:00,  3.96img/s, loss (batch)=0.0793]
INFO: Checkpoint 30 saved at 10_16_2021_09_00_52!
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.10batch/s]
                                                                    INFO: Validation Dice score: 0.6303732395172119
Validation round:  96%|█████████▌| 22/23 [00:01<00:00, 18.19batch/s]
                                                                    INFO: Validation Dice score: 0.53330892324

Traceback (most recent call last):
  File "<ipython-input-16-4a170b73a195>", line 8, in <module>
    amp=amp)
  File "<ipython-input-10-794b113d5a00>", line 97, in train_net
    val_score = evaluate(net, val_loader, device)
  File "/data/clothing-size-identification/utils/evaluate.py", line 31, in evaluate
    mask_pred = F.one_hot(mask_pred.argmax(dim=1), net.n_classes).permute(0, 3, 1, 2).float()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-4a170b73a195>", line 12, in <module>
    sys.exit(0)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 148, in exit
    self._orig_exit(orig_code)
  File "/usr/local/lib/python3.6/dist-packages/wandb/sdk/wandb_run.py", line 148, in exit
    self._orig_exit(orig_c

TypeError: object of type 'NoneType' has no len()